# Chords Progressions Transformer (ver. 1.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

WARNING: This complete implementation is a functioning model of the Artificial Intelligence. Please excercise great humility, care, and respect. https://www.nscai.gov/

***

#### Project Los Angeles

#### Tegridy Code 2024

***

# (GPU CHECK)

In [ ]:
#@title NVIDIA GPU check
!nvidia-smi

# (SETUP ENVIRONMENT)

In [ ]:
#@title Install dependencies
!git clone --depth 1 https://github.com/asigalov61/Chords-Progressions-Transformer
!pip install huggingface_hub
!pip install einops
!pip install torch-summary
!apt install fluidsynth #Pip does not work for some reason. Only apt works

In [ ]:
#@title Import modules

print('=' * 70)
print('Loading core Chords Progressions Transformer modules...')

import os
import copy
import pickle
import secrets
import statistics
from time import time
import tqdm

print('=' * 70)
print('Loading main Chords Progressions Transformer modules...')
import torch

%cd /content/Chords-Progressions-Transformer

import TMIDIX

from midi_to_colab_audio import midi_to_colab_audio

from x_transformer_1_23_2 import *

import random

%cd /content/
print('=' * 70)
print('Loading aux Chords Progressions Transformer modules...')

import matplotlib.pyplot as plt

from torchsummary import summary
from sklearn import metrics

from IPython.display import Audio, display

from huggingface_hub import hf_hub_download

from google.colab import files

print('=' * 70)
print('Done!')
print('Enjoy! :)')
print('=' * 70)

# (LOAD MODEL)

In [ ]:
#@title Load Chords Progressions Transformer Pre-Trained Model

#@markdown Choose model

select_model_to_load = "187M-2048E-4L-16H-FP32-Fast-Small" # @param ["93M-1024E-8L-8H-Very-Fast-Small", "187M-2048E-4L-16H-Fast-Small", "187M-2048E-4L-16H-FP32-Fast-Small"]

#@markdown Model precision option

model_precision = "bfloat16" # @param ["bfloat16", "float16", "float32"]

#@markdown bfloat16 == Half precision/faster speed (if supported, otherwise the model will default to float16)

#@markdown float16 == Full precision/fast speed

plot_tokens_embeddings = False # @param {type:"boolean"}

print('=' * 70)
print('Loading Chords Progressions Transformer Pre-Trained Model...')
print('Please wait...')
print('=' * 70)

full_path_to_models_dir = "/content/Chords-Progressions-Transformer/Models"

if select_model_to_load == '93M-1024E-8L-8H-Very-Fast-Small':

  dim = 1024
  depth = 8
  heads = 8

  model_checkpoint_file_name = 'Chords_Progressions_Transformer_Small_Trained_Model_9609_steps_1.0704_loss_0.6927_acc.pth'
  model_path = full_path_to_models_dir+'/Small/'+model_checkpoint_file_name
  if os.path.isfile(model_path):
    print('Model already exists...')

  else:
    hf_hub_download(repo_id='asigalov61/Chords-Progressions-Transformer',
                    filename=model_checkpoint_file_name,
                    local_dir='/content/Chords-Progressions-Transformer/Models/Small',
                    local_dir_use_symlinks=False)

elif select_model_to_load == '187M-2048E-4L-16H-Fast-Small':

  dim = 2048
  depth = 4
  heads = 16

  model_checkpoint_file_name = 'Chords_Progressions_Transformer_Small_2048_Trained_Model_12947_steps_0.9316_loss_0.7386_acc.pth'
  model_path = full_path_to_models_dir+'/Small_2048/'+model_checkpoint_file_name
  if os.path.isfile(model_path):
    print('Model already exists...')

  else:
    hf_hub_download(repo_id='asigalov61/Chords-Progressions-Transformer',
                    filename=model_checkpoint_file_name,
                    local_dir='/content/Chords-Progressions-Transformer/Models/Small_2048',
                    local_dir_use_symlinks=False)

elif select_model_to_load == '187M-2048E-4L-16H-FP32-Fast-Small':

  dim = 2048
  depth = 4
  heads = 16

  model_checkpoint_file_name = 'Chords_Progressions_Transformer_Small_2048_FP32_Trained_Model_6265_steps_0.9272_loss_0.7369_acc.pth'
  model_path = full_path_to_models_dir+'/Small_2048_FP32/'+model_checkpoint_file_name
  if os.path.isfile(model_path):
    print('Model already exists...')

  else:
    hf_hub_download(repo_id='asigalov61/Chords-Progressions-Transformer',
                    filename=model_checkpoint_file_name,
                    local_dir='/content/Chords-Progressions-Transformer/Models/Small_2048_FP32',
                    local_dir_use_symlinks=False)

print('=' * 70)
print('Instantiating model...')

device_type = 'cuda'

if model_precision == 'bfloat16' and torch.cuda.is_bf16_supported():
  dtype = 'bfloat16'
else:
  dtype = 'float16'

if model_precision == 'float16':
  dtype = 'float16'

if model_precision == 'float32':
  dtype = 'float32'

ptdtype = {'bfloat16': torch.bfloat16, 'float16': torch.float16, 'float32': torch.float32}[dtype]
ctx = torch.amp.autocast(device_type=device_type, dtype=ptdtype)

SEQ_LEN = 8192 # Models seq len
PAD_IDX = 707 # Models pad index

# instantiate the model

model = TransformerWrapper(
    num_tokens = PAD_IDX+1,
    max_seq_len = SEQ_LEN,
    attn_layers = Decoder(dim = dim, depth = depth, heads = heads, attn_flash = True)
    )

model = AutoregressiveWrapper(model, ignore_index = PAD_IDX)

model.cuda()
print('=' * 70)

print('Loading model checkpoint...')

model.load_state_dict(torch.load(model_path))
print('=' * 70)

model.eval()

print('Done!')
print('=' * 70)

print('Model will use', dtype, 'precision...')
print('=' * 70)

# Model stats
print('Model summary...')
summary(model)

# Plot Token Embeddings
if plot_tokens_embeddings:
  tok_emb = model.net.token_emb.emb.weight.detach().cpu().tolist()

  cos_sim = metrics.pairwise_distances(
    tok_emb, metric='cosine'
  )
  plt.figure(figsize=(7, 7))
  plt.imshow(cos_sim, cmap="inferno", interpolation="nearest")
  im_ratio = cos_sim.shape[0] / cos_sim.shape[1]
  plt.colorbar(fraction=0.046 * im_ratio, pad=0.04)
  plt.xlabel("Position")
  plt.ylabel("Position")
  plt.tight_layout()
  plt.plot()
  plt.savefig("/content/Chords-Progressions-Transformer-Tokens-Embeddings-Plot.png", bbox_inches="tight")

# (GENERATE)

# (IMPROV)

In [ ]:
#@title Standard Improv Generator

#@markdown Generation settings

output_MIDI_patch_number = 0 # @param {type:"slider", min:0, max:127, step:1}
number_of_tokens_tp_generate = 258 # @param {type:"slider", min:30, max:8190, step:3}
number_of_batches_to_generate = 4 #@param {type:"slider", min:1, max:16, step:1}
temperature = 0.9 # @param {type:"slider", min:0.1, max:1, step:0.05}

#@markdown Other settings

render_MIDI_to_audio = True # @param {type:"boolean"}

print('=' * 70)
print('Chords Progressions Transformer Standard Improv Model Generator')
print('=' * 70)

start_chord = random.randint(0, 320)

outy = [start_chord+384, 0]

print('Start chord:', TMIDIX.ALL_CHORDS_SORTED[start_chord])
print('=' * 70)
print('Selected Improv sequence:')
print(outy)
print('=' * 70)

torch.cuda.empty_cache()

inp = [outy] * number_of_batches_to_generate

inp = torch.LongTensor(inp).cuda()

with ctx:
  out = model.generate(inp,
                        number_of_tokens_tp_generate,
                        temperature=temperature,
                        return_prime=True,
                        verbose=True)

out0 = out.tolist()

print('=' * 70)
print('Done!')
print('=' * 70)

torch.cuda.empty_cache()

#======================================================================

print('Rendering results...')

for i in range(number_of_batches_to_generate):

  print('=' * 70)
  print('Batch #', i)
  print('=' * 70)

  out1 = out0[i]

  print('Sample INTs', out1[:12])
  print('=' * 70)

  if len(out1) != 0:

      song = out1
      song_f = []

      time = 0
      dur = 0
      vel = 90
      pitch = 0
      channel = 0

      patches = [0] * 16
      patches[0] = output_MIDI_patch_number

      for ss in song:

          if 0 <= ss < 128:

              time += ss * 32

          if 128 <= ss < 256:

              dur = (ss-128) * 32

          if 256 <= ss < 384:

              pitch = (ss-256)

              vel = max(40, pitch)

              song_f.append(['note', time, dur, channel, pitch, vel, output_MIDI_patch_number])

      detailed_stats = TMIDIX.Tegridy_ms_SONG_to_MIDI_Converter(song_f,
                                                              output_signature = 'Chords Progressions Transformer',
                                                              output_file_name = '/content/Chords-Progressions-Transformer-Composition_'+str(i),
                                                              track_name='Project Los Angeles',
                                                              list_of_MIDI_patches=patches
                                                              )

      print('=' * 70)
      print('Displaying resulting composition...')
      print('=' * 70)

      fname = '/content/Chords-Progressions-Transformer-Composition_'+str(i)

      if render_MIDI_to_audio:
        midi_audio = midi_to_colab_audio(fname + '.mid')
        display(Audio(midi_audio, rate=16000, normalize=False))

      TMIDIX.plot_ms_SONG(song_f, plot_title=fname)

# (CUSTOM MIDI)

In [ ]:
#@title Load Seed MIDI

#@markdown Press play button to to upload your own seed MIDI or to load one of the provided sample seed MIDIs from the dropdown list below

select_seed_MIDI = "Upload your own custom MIDI" # @param ["Upload your own custom MIDI", "Chords-Progressions-Transformer-Piano-Seed-1", "Chords-Progressions-Transformer-Piano-Seed-2", "Chords-Progressions-Transformer-Piano-Seed-3", "Chords-Progressions-Transformer-Piano-Seed-4", "Chords-Progressions-Transformer-Piano-Seed-5", "Chords-Progressions-Transformer-Piano-Seed-6", "Chords-Progressions-Transformer-MI-Seed-1", "Chords-Progressions-Transformer-MI-Seed-2", "Chords-Progressions-Transformer-MI-Seed-3", "Chords-Progressions-Transformer-MI-Seed-4", "Chords-Progressions-Transformer-MI-Seed-5", "Chords-Progressions-Transformer-MI-Seed-6"]
strip_notes_from_composition = False # @param {type:"boolean"}
render_MIDI_to_audio = False # @param {type:"boolean"}

print('=' * 70)
print('Chords Progressions Transformer Seed MIDI Loader')
print('=' * 70)

f = ''

if select_seed_MIDI != "Upload your own custom MIDI":
  print('Loading seed MIDI...')
  f = '/content/Chords-Progressions-Transformer/Seeds/'+select_seed_MIDI+'.mid'

else:
  print('Upload your own custom MIDI...')
  print('=' * 70)
  uploaded_MIDI = files.upload()
  if list(uploaded_MIDI.keys()):
    f = list(uploaded_MIDI.keys())[0]

if f != '':

  print('=' * 70)
  print('File:', f)
  print('=' * 70)

  #=======================================================
  # START PROCESSING

  #===============================================================================
  # Raw single-track ms score

  raw_score = TMIDIX.midi2single_track_ms_score(f)

  #===============================================================================
  # Enhanced score notes

  escore_notes = TMIDIX.advanced_score_processor(raw_score, return_enhanced_score_notes=True)[0]

  no_drums_escore_notes = [e for e in escore_notes if e[6] < 80]

  if len(no_drums_escore_notes) > 0:

    #=======================================================
    # PRE-PROCESSING

    #===============================================================================
    # Augmented enhanced score notes

    if select_model_to_load == '187M-2048E-4L-16H-FP32-Fast-Small':
      no_drums_escore_notes = TMIDIX.augment_enhanced_score_notes(no_drums_escore_notes,
                                                                  timings_divider=32
                                                                  )
    else:
      no_drums_escore_notes = TMIDIX.augment_enhanced_score_notes(no_drums_escore_notes)

    cscore = TMIDIX.chordify_score([1000, no_drums_escore_notes])

    clean_cscore = []

    for c in cscore:
        pitches = []
        cho = []
        for cc in c:
            if cc[4] not in pitches:
                cho.append(cc)
                pitches.append(cc[4])

        clean_cscore.append(cho)

    #=======================================================
    # FINAL PROCESSING

    melody_chords = []
    chords = []
    times = [0]
    durs = []

    #=======================================================
    # MAIN PROCESSING CYCLE
    #=======================================================

    pe = clean_cscore[0][0]

    first_chord = True

    for c in clean_cscore:

        # Chords

        c.sort(key=lambda x: x[4], reverse=True)

        tones_chord = sorted(set([cc[4] % 12 for cc in c]))

        try:
            chord_token = TMIDIX.ALL_CHORDS_SORTED.index(tones_chord)
        except:
            checked_tones_chord = TMIDIX.check_and_fix_tones_chord(tones_chord)
            chord_token = TMIDIX.ALL_CHORDS_SORTED.index(checked_tones_chord)

        melody_chords.extend([chord_token+384])

        if strip_notes_from_composition:
          if len(tones_chord) > 1:
            chords.extend([chord_token+384])

        else:
          chords.extend([chord_token+384])

        if first_chord:
                melody_chords.extend([0])
                first_chord = False

        for e in c:

            #=======================================================
            # Timings...

            time = e[1]-pe[1]

            dur = e[2]

            if select_model_to_load == '187M-2048E-4L-16H-FP32-Fast-Small':

              delta_time = max(0, min(127, time))
              dur = max(1, min(127, dur))

            else:

              if time != 0 and time % 2 != 0:
                  time += 1

              if dur % 2 != 0:
                  dur += 1

              delta_time = int(max(0, min(255, time)) / 2)

              dur = int(max(0, min(255, dur)) / 2)

            # Pitches

            ptc = max(1, min(127, e[4]))

            #=======================================================
            # FINAL NOTE SEQ

            # Writing final note asynchronously

            if delta_time != 0:
                melody_chords.extend([delta_time, dur+128, ptc+256])
                if strip_notes_from_composition:
                  if len(c) > 1:
                    times.append(delta_time)
                    durs.append(dur+128)
                else:
                    times.append(delta_time)
                    durs.append(dur+128)
            else:
                melody_chords.extend([dur+128, ptc+256])

            pe = e

    #=======================================================

    # TOTAL DICTIONARY SIZE 706+1=707
    #=======================================================

  #=======================================================

  song = melody_chords
  song_f = []

  time = 0
  dur = 0
  vel = 90
  pitch = 0
  channel = 0

  patches = [0] * 16

  channel = 0

  for ss in song:

      if 0 <= ss < 128:

          time += ss * 32

      if 128 <= ss < 256:

          dur = (ss-128) * 32

      if 256 <= ss < 384:

          pitch = (ss-256)

          vel = max(40, pitch)

          song_f.append(['note', time, dur, channel, pitch, vel, 0])

  detailed_stats = TMIDIX.Tegridy_ms_SONG_to_MIDI_Converter(song_f,
                                                          output_signature = 'Chords Progressions Transformer',
                                                          output_file_name = '/content/Chords-Progressions-Transformer-Seed-Composition',
                                                          track_name='Project Los Angeles',
                                                          list_of_MIDI_patches=patches
                                                          )

  #=======================================================

  print('=' * 70)
  print('Composition stats:')
  print('Composition has', len(no_drums_escore_notes), 'notes')
  print('Composition has', len(chords), 'chords')
  print('Composition has', len(melody_chords), 'tokens')
  print('=' * 70)

  print('Displaying resulting composition...')
  print('=' * 70)

  fname = '/content/Chords-Progressions-Transformer-Seed-Composition'

  if render_MIDI_to_audio:
    midi_audio = midi_to_colab_audio(fname + '.mid')
    display(Audio(midi_audio, rate=16000, normalize=False))

  TMIDIX.plot_ms_SONG(song_f, plot_title=fname)

else:
  print('=' * 70)

# (CONTINUATIONS)

In [ ]:
#@title Standard Continuation

#@markdown Generation settings

output_MIDI_patch_number = 0 # @param {type:"slider", min:0, max:127, step:1}
number_of_prime_tokens = 512 # @param {type:"slider", min:4, max:8192, step:4}
number_of_tokens_to_generate = 512 # @param {type:"slider", min:40, max:8192, step:4}
number_of_batches_to_generate = 4 #@param {type:"slider", min:1, max:16, step:1}
temperature = 0.9 # @param {type:"slider", min:0.1, max:1, step:0.05}

#@markdown Other settings
include_prime_tokens_in_generated_output = True #@param {type:"boolean"}
render_MIDI_to_audio = True # @param {type:"boolean"}

print('=' * 70)
print('Chords Progressions Transformer Standard Continuation Model Generator')
print('=' * 70)

outy = melody_chords[:number_of_prime_tokens]

torch.cuda.empty_cache()

inp = [outy] * number_of_batches_to_generate

inp = torch.LongTensor(inp).cuda()

with ctx:
  out = model.generate(inp,
                        number_of_tokens_to_generate,
                        temperature=temperature,
                        return_prime=include_prime_tokens_in_generated_output,
                        verbose=True)

out0 = out.tolist()

torch.cuda.empty_cache()

print('=' * 70)
print('Done!')
print('=' * 70)

#======================================================================
print('Rendering results...')

for i in range(number_of_batches_to_generate):

  print('=' * 70)
  print('Batch #', i)
  print('=' * 70)

  out1 = out0[i]

  print('Sample INTs', out1[:12])
  print('=' * 70)

  if len(out) != 0:

      song = out1
      song_f = []

      time = 0
      dur = 0
      vel = 90
      pitch = 0
      channel = 0

      patches = [0] * 16
      patches[0] = output_MIDI_patch_number

      for ss in song:

          if 0 <= ss < 128:

              time += ss * 32

          if 128 <= ss < 256:

              dur = (ss-128) * 32

          if 256 <= ss < 384:

              pitch = (ss-256)

              vel = max(40, pitch)

              song_f.append(['note', time, dur, channel, pitch, vel, output_MIDI_patch_number])

      detailed_stats = TMIDIX.Tegridy_ms_SONG_to_MIDI_Converter(song_f,
                                                              output_signature = 'Chords Progressions Transformer',
                                                              output_file_name = '/content/Chords-Progressions-Transformer-Composition_'+str(i),
                                                              track_name='Project Los Angeles',
                                                              list_of_MIDI_patches=patches
                                                              )



      print('=' * 70)
      print('Displaying resulting composition...')
      print('=' * 70)

      fname = '/content/Chords-Progressions-Transformer-Composition_'+str(i)

      if render_MIDI_to_audio:
        midi_audio = midi_to_colab_audio(fname + '.mid')
        display(Audio(midi_audio, rate=16000, normalize=False))

      TMIDIX.plot_ms_SONG(song_f, plot_title=fname)

# (CHORDS PROGRESSIONS)

In [ ]:
# @title Generate chords progressions from custom MIDI chords

#@markdown NOTE: You can stop the generation at any time to render partial results

#@markdown Generation settings

conditioning_type = "Chords" # @param ["Chords", "Chords-Times", "Chords-Times-Durations"]
output_MIDI_patch_number = 0 # @param {type:"slider", min:0, max:127, step:1}
number_of_chords_to_generate = 128 # @param {type:"slider", min:8, max:4096, step:1}
max_number_of_notes_per_chord = 8 # @param {type:"slider", min:1, max:10, step:1}
number_of_memory_tokens = 4096 # @param {type:"slider", min:32, max:8188, step:4}
temperature = 0.9 # @param {type:"slider", min:0.1, max:1, step:0.05}

#@markdown Other settings

render_MIDI_to_audio = True # @param {type:"boolean"}

#===============================================================================

def generate_chords(chords,
                    max_chords_limit = 8,
                    num_memory_tokens = 4096,
                    temperature=0.9
                    ):

    chords = chords[-num_memory_tokens:]

    x = torch.tensor([chords] * 1, dtype=torch.long, device='cuda')

    o = 0

    ncount = 0

    while o < 384 and ncount < max_chords_limit:
      with ctx:
        out = model.generate(x,
                            1,
                            temperature=temperature,
                            return_prime=False,
                            verbose=False)

      o = out.tolist()[0][0]

      if 256 <= o < 384:
        ncount += 1

      if o < 384:
        x = torch.cat((x, out), 1)

    return x.tolist()[0][len(chords):]


#===============================================================================

print('=' * 70)
print('Chords Progressions Model Generator')
print('=' * 70)

torch.cuda.empty_cache()

output = []

idx = 0

for c in tqdm.tqdm(chords[:number_of_chords_to_generate]):

  try:

    output.append(c)

    if conditioning_type == 'Chords-Times' or conditioning_type == 'Chords-Times-Durations':
      output.append(times[idx])

    if conditioning_type == 'Chords-Times-Durations':
      output.append(durs[idx])

    out = generate_chords(output,
                        temperature=temperature,
                        max_chords_limit=max_number_of_notes_per_chord,
                        num_memory_tokens=number_of_memory_tokens
                        )
    output.extend(out)

    idx += 1

  except KeyboardInterrupt:
    print('=' * 70)
    print('Stopping generation...')
    break

  except Exception as e:
    print('=' * 70)
    print('Error:', e)
    break

torch.cuda.empty_cache()

#===============================================================================
print('=' * 70)

out1 = output

print('Sample INTs', out1[:12])
print('=' * 70)

if len(out) != 0:

    song = out1
    song_f = []

    time = 0
    dur = 0
    vel = 90
    pitch = 0
    channel = 0

    patches = [0] * 16
    patches[0] = output_MIDI_patch_number

    for ss in song:

        if 0 <= ss < 128:

            time += ss * 32

        if 128 <= ss < 256:

            dur = (ss-128) * 32

        if 256 <= ss < 384:

            pitch = (ss-256)

            vel = max(40, pitch)

            song_f.append(['note', time, dur, channel, pitch, vel, output_MIDI_patch_number])

    detailed_stats = TMIDIX.Tegridy_ms_SONG_to_MIDI_Converter(song_f,
                                                            output_signature = 'Chords Progressions Transformer',
                                                            output_file_name = '/content/Chords-Progressions-Transformer-Composition',
                                                            track_name='Project Los Angeles',
                                                            list_of_MIDI_patches=patches
                                                            )



    print('=' * 70)
    print('Displaying resulting composition...')
    print('=' * 70)

    fname = '/content/Chords-Progressions-Transformer-Composition'

    if render_MIDI_to_audio:
      midi_audio = midi_to_colab_audio(fname + '.mid')
      display(Audio(midi_audio, rate=16000, normalize=False))

    TMIDIX.plot_ms_SONG(song_f, plot_title=fname)

# Congrats! You did it! :)